# Continued pretraining (CP) using Unsloth

In [ ]:
#install latest version of unsloth
#%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

#%pip install --upgrade unsloth
# %pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
# %pip install --upgrade unsloth_zoo
# %pip install wandb

In [12]:
%pip install --upgrade  wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 41.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.19.1
    Uninstalling wandb-0.19.1:
      Successfully uninstalled wandb-0.19.1
  You can safely remove it manually.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install --upgrade unsloth unsloth_zoo

  Using cached torch-2.5.1-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached triton-3.1.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
Using cached torch-2.5.1-cp312-cp312-manylinux1_x86_64.whl (906.4 MB)
Using cached triton-3.1.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 85.2 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0
    Uninstalling torch-2.6.0:
      Successfully uninstalled torch-2.6.0
  Attempting uninstall: unsloth_zoo
    Found exi

In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported

from unsloth import UnslothTrainingArguments, UnslothTrainer
from datasets import load_dataset

import torch

# random state
SEED = 42

# reproducibility
## torch
torch.manual_seed(SEED)

## python
import random
random.seed(SEED)

## numpy
import numpy as np
np.random.seed(SEED)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## 🧠 Load Model

In [2]:
max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model_id = "meta-llama/Llama-3.1-8B"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.3: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.254 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Init LoRA

In [3]:
from peft import LoftQConfig

"""
    This is the sub-configuration class to store the configuration of a [`LoraModel`].

    Args:
        bits_pattern (`dict`): The mapping from layer names or regexp expression to bits which are different from the
            default bits specified by `bits`. For example, `{model.decoder.layers.0.encoder_attn.k_proj: 2`}.
        bits (`int`): Quantization bits for LoftQ.
        iter (`int`): Alternating iterations for LoftQ.
        fake (`bool`): True: use fp16/fp32; used for first time to save weights. False: use bitsandbytes 4bit linear
            models. weights can't be saved. Recommend to set to True, save the weights and load the saved weights in 4
            bits.
    """

loftq_config = LoftQConfig(loftq_bits=4)

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 256, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head"], #now also embeddings and lm_head
    lora_alpha = 1,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = SEED,
    use_rslora = True,  # We support rank stabilized LoRA
    init_lora_weights = "loftq",
    loftq_config = loftq_config, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.3.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


## Prepare Data

In [5]:
# from datasets import load_dataset

# # this dataset has already fixed encoding using ftfy (as is used by me in the preprocessing steps of other datasets)
dataset_cs = load_dataset("HuggingFaceFW/fineweb-2", "ces_Latn", split="train", streaming=True)
#we need only texts
dataset_cs = dataset_cs.remove_columns(["id", "dump", "url", "date", "file_path", "language", "language_score", "language_script", "minhash_cluster_size", "top_langs"])
#shuffle to be sure we select "random sample"
dataset_cs = dataset_cs.shuffle(seed=42)
dataset_cs

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

IterableDataset({
    features: ['text'],
    n_shards: 25
})

In [6]:
dataset_cs = dataset_cs.select(range(19531250))
dataset_cs

AttributeError: 'IterableDataset' object has no attribute 'select'

In [18]:
19531250/1024

19073.486328125

In [7]:
dataset_cs = dataset_cs.take(19531250)
dataset_cs

IterableDataset({
    features: ['text'],
    n_shards: 25
})

In [10]:
x=10000000000/512
x

19531250.0

In [8]:
len(dataset_cs)

TypeError: object of type 'IterableDataset' has no len()

In [18]:
num_all_tokens = len(dataset_cs)*512 #number of tokens when using 512 tokens per data point
num_all_tokens

32104170496

In [14]:
#number of tokens per batch (processed in one step)
global_batch_size = 1024

tokens_per_step = global_batch_size*max_seq_length
tokens_per_step

524288

In [17]:
#number of steps to process 10 B tokens
num_steps = 10e9/tokens_per_step
num_steps

19073.486328125

In [16]:
#determine step number when len of dataset is not available when streaming
len(dataset_cs) / 1024

61233.845703125

In [ ]:
# dataset_en = load_dataset("HuggingFaceFW/fineweb-edu", "sample-10BT", split="train")
# dataset_en


In [ ]:
# dataset_en = dataset_en.remove_columns(["id", "dump", "url", "file_path", "language", "language_score", "token_count", "score", "int_score"])
# dataset_en.shuffle(seed=42)
# dataset_en

In [ ]:
# from datasets import concatenate_datasets
# #select 40000 samples (for the initial exp.)

# dataset_cs = dataset_cs.select(range(30000))
# dataset_en = dataset_en.select(range(10000))

# dataset = concatenate_datasets([dataset_cs, dataset_en])
# dataset

In [ ]:
# dataset.save_to_disk("data/pretraining/init_31mix_cs-en")

In [ ]:
# from datasets import load_from_disk
# dataset = load_from_disk("data/pretraining/init_31mix_cs-en")
# dataset

Dataset({
    features: ['text'],
    num_rows: 40000
})

In [9]:
dataset = dataset_cs

In [10]:
dataset

IterableDataset({
    features: ['text'],
    n_shards: 25
})

In [11]:
def preprocess_function(examples):
    return {"text": [example + tokenizer.eos_token for example in examples["text"]]}

dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
# for row in dataset[:5]["text"]:
#     print("=========================")
#     print(row)

## Load Prepared Data

In [6]:
from datasets import load_from_disk, load_dataset

dataset = load_from_disk("data/pretraining/fineweb-2_ces_Latn_19531250_llama_preprocessed")
#dataset = dataset.select(range(1000000))
dataset = dataset.to_iterable_dataset()
dataset

Loading dataset from disk:   0%|          | 0/134 [00:00<?, ?it/s]

IterableDataset({
    features: ['text'],
    n_shards: 1
})

## Train the model

### Trainer Modifications

In [ ]:
class UnslothKLLossTrainer(UnslothTrainer):
    def __init__(self, ref_model, kl_weight=0.1, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ref_model = ref_model
        self.ref_model.eval()
        self.kl_weight = kl_weight

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        #account for the actual number of items in the batch
        if num_items_in_batch is not None:
            effective_batch_size = num_items_in_batch
        else:
            effective_batch_size = labels.size(0)
        
        #cross entropy loss
        loss_fct = torch.nn.CrossEntropyLoss(reduction="sum")
        loss_ce = loss_fct(logits.view(-1, logits.shape[-1]), labels.view(-1))
        loss_ce = loss_ce / effective_batch_size # Scale by actual number of items

        # KL loss
        with torch.no_grad():
            ref_outputs = self.ref_model(**inputs)
            ref_logits = ref_outputs.logits
        
        loss_kl_fct = torch.nn.KLDivLoss(reduction="sum")
        loss_kl = loss_kl_fct(
            torch.nn.functional.log_softmax(logits, dim=-1),
            torch.nn.functional.softmax(ref_logits, dim=-1),
        )
        loss_kl = loss_kl / effective_batch_size # Scale by actual number of items

        total_loss = loss_ce + self.kl_weight*loss_kl

        return (total_loss, outputs) if return_outputs else total_loss

from transformers.utils.import_utils import is_torch_xla_available
#from transformers.trainer_utils import SaveStrategy
from typing import Dict
import numpy as np

class PerplexityTrainer(UnslothTrainer):
    # def training_step(self, model, inputs, num_items_in_batch):
    #     loss = super().training_step(model, inputs, num_items_in_batch)

    #     # log perplexity
    #     perplexity = torch.exp(loss)
    #     self.log({"train_perplexity": perplexity})

    #     return loss
    def _maybe_log_save_evaluate(self, tr_loss, grad_norm, model, trial, epoch, ignore_keys_for_eval):
        if self.control.should_log and self.state.global_step > self._globalstep_last_logged:
            if is_torch_xla_available():
                xm.mark_step()

            logs: Dict[str, float] = {}

            # all_gather + mean() to get average loss over all processes
            tr_loss_scalar = self._nested_gather(tr_loss).mean().item()

            # reset tr_loss to zero
            tr_loss -= tr_loss

            loss = round(tr_loss_scalar / (self.state.global_step - self._globalstep_last_logged), 4)
            logs["loss"] = loss

            #PERPLEXITY
            logs["perplexity"] = round(float(np.exp(tr_loss_scalar)), 4)
            #########################


            if grad_norm is not None:
                logs["grad_norm"] = grad_norm.detach().item() if isinstance(grad_norm, torch.Tensor) else grad_norm
            logs["learning_rate"] = self._get_learning_rate()

            self._total_loss_scalar += tr_loss_scalar
            self._globalstep_last_logged = self.state.global_step
            self.store_flos()

            self.log(logs)

        metrics = None
        if self.control.should_evaluate:
            metrics = self._evaluate(trial, ignore_keys_for_eval)

        if self.control.should_save:
            self._save_checkpoint(model, metrics=metrics)
            self.control = self.callback_handler.on_save(self.args, self.state, self.control)

TrainerClass = PerplexityTrainer

In [9]:
from transformers import EvalPrediction
import numpy as np

#prepare perplexity computation
def compute_metrics(pred: EvalPrediction):
    losses = pred.losses

    mean_loss = np.mean(losses)

    try:
        perplexity = np.exp(mean_loss)
    except OverflowError:
        perplexity = float("inf")

    return {"perplexity": perplexity}


### Training

In [10]:
trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    # ref_model=copy.deepcopy(model),
    #compute_metrics=compute_metrics,
    
    args = UnslothTrainingArguments(
        per_device_train_batch_size = 256,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.05,
        #num_train_epochs = 1, # Set this for 1 full training run. - not supported for streaming -> use max_steps (here 61234=dataset_len/global_batch_size)
        max_steps = 19000,
        learning_rate = 5e-4,
        embedding_learning_rate = 5e-4/2,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = SEED,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        #run_name="llama3.2-1b-cp-exp-loftq",
        # eval_strategy = "steps",
        # eval_steps = 50,
        
    ),
)

/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:578: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:592: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:606: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [11]:
#Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-80GB. Max memory = 79.254 GB.
19.811 GB of memory reserved.


In [11]:
!export WANDB_DISABLE_SSL=true

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,456,000 | Num Epochs = 9,223,372,036,854,775,807 | Total steps = 19,000
O^O/ \_/ \    Batch size per device = 256 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (256 x 4 x 1) = 1,024
 "-____-"     Trainable parameters = 1,721,761,792/9,752,023,040 (17.66% trained)


Unsloth: Setting lr = 2.50e-04 instead of 5.00e-04 for embed_tokens.
Unsloth: Setting lr = 2.50e-04 instead of 5.00e-04 for lm_head.
Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.331100


In [15]:
#Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4329.0353 seconds used for training.
72.15 minutes used for training.
Peak reserved memory = 19.932 GB.
Peak reserved memory for training = 17.297 GB.
Peak reserved memory % of max memory = 50.613 %.
Peak reserved memory for training % of max memory = 43.922 %.
